In [ ]:
import ortools
from ortools.constraint_solver import routing_enums_pb2
import ortools.constraint_solver.pywrapcp as pywrapcp
from PIL import Image
from IPython.display import display
import pandas as pd
from itertools import zip_longest
from geopy.distance import geodesic

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import requests

import folium

import seaborn as sns

import itertools
import math

# Load do DataFrame e preprocessamento

In [ ]:
# caminho = r"ara_adm (1) - Copia.xlsx".replace("\\","/")
caminho = r"ara_adm (1).xlsx".replace("\\","/")
teste = pd.read_excel(caminho)
teste = teste.drop_duplicates()
teste = teste.reset_index()
teste['lat'] = teste['lat'].astype(str)
teste['long'] = teste['long'].astype(str)
teste['lat'] = teste['lat'].str.replace(",",".")
teste['long'] = teste['long'].str.replace(",",".")
teste['lat'] = teste['lat'].astype(float)
teste['long'] = teste['long'].astype(float)
teste['qtd'] = teste['qtd'].astype(int)

teste

In [ ]:
def verificar_coluna(dataframe, nome_coluna):
    if nome_coluna in dataframe.columns:
        return True
    else:
        return False

# Verificar se a coluna 'start' existe

#### Abaixo a variavel **initialize_starting_points** é a variável vai controlar o fluxo do código, para definir o modo de inicialização das próximas variáveis

In [ ]:
if verificar_coluna(teste, 'start') == True:
    teste['start'] = teste['start'].astype(str)
    teste['start'] = teste['start'].str.strip()
    teste['start'] = teste['start'].str.lower()
    initialize_starting_points = False
else:
    starts = None
    n_routs_desirable = None
    initialize_starting_points = True

# Funções que serão ultilizadas no código

A "distance_matrix" (matriz de distâncias) no contexto de roteamento de veículos, especialmente com a biblioteca ortools da Google, é uma representação matricial das distâncias entre diferentes locais em uma rede viária. Essa matriz é fundamental para resolver problemas de otimização de rotas, ajudando a determinar a distância mais curta ou o tempo de viagem entre todos os pares de locais na rede.

In [ ]:
def create_distance_matrix(DataFrame = pd.DataFrame(), lat=str(), long=str()):
    x = list(zip_longest(DataFrame['lat'][:], DataFrame['long'][:], fillvalue=None))
    # Crie uma matriz de distância vazia
    distance_matrix = []

    # Calcule as distâncias entre todos os pares de coordenadas
    for coord1 in x:
        distances = []
        for coord2 in x:
            distancia = geodesic(coord1, coord2).kilometers
            distances.append(distancia)
        distance_matrix.append(distances)
        
    return distance_matrix

In [ ]:
def create_data_frame(list_list = list(), list_str = list()):
    DataFrame = pd.DataFrame()
    if len(list_list) != len(list_str):
        return 'len(list_list) != len(list_str)'
    else:
        for i in range(len(list_list)):
            DataFrame[list_str[i]] = list_list[i]
        
        return DataFrame

In [ ]:
def capturate_start_points(common_column, DataFrame_1 = pd.DataFrame(), DataFrame_2 = pd.DataFrame()):
    if isinstance(common_column, list):
        DataFrame = pd.merge(DataFrame_1, DataFrame_2, on = common_column)
    elif isinstance(objeto, str):
        DataFrame = pd.merge(DataFrame_1, DataFrame_2, on = common_column)
        
    return DataFrame

In [ ]:
def cluster_data(data_frame, n_clusters):
    # Selecionar as colunas 'lat' e 'long'
    features = data_frame[['lat', 'long']]

    # Padronizar as features para média zero e desvio padrão 1
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Executar o algoritmo K-Means com o número de grupos especificado
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    data_frame['cluster'] = kmeans.fit_predict(scaled_features)
    data_frame = data_frame.assign(row=1)

    return data_frame

In [ ]:
def plot_clusters_on_map(data_frame, lat_col, long_col, cluster_col):
    # Extrair as coordenadas de latitude e longitude e os rótulos de grupo
    lats = data_frame[lat_col]
    longs = data_frame[long_col]
    clusters = data_frame[cluster_col]

    # Configurar o estilo do gráfico de dispersão
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 8))

    # Plotar os pontos coloridos por grupo
    sns.scatterplot(x=longs, y=lats, hue=clusters, palette='Set1', s=100, legend='full')

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Grupos de Dados no Mapa')

    # Mostrar a legenda
    plt.legend(title='Grupos', loc='upper right')

    # Calcular e plotar os centroides dos clusters
    centroids = data_frame.groupby(cluster_col)[[lat_col, long_col]].mean().reset_index()
    sns.scatterplot(x=centroids[long_col], y=centroids[lat_col], hue=centroids[cluster_col],
                    palette='Set1', s=200, marker='X', legend=False, ax=plt.gca())

    # Exibir o mapa
    plt.show()

    return centroids

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Raio da Terra em quilômetros
    radius = 6371

    # Converte graus para radianos
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Diferenças de latitude e longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c

    return distance

In [ ]:
def calcular_proporcionais(valores, valor_base):
    # Verifica se a lista de valores não está vazia
    if not valores:
        return []

    # Calcula a soma dos valores na lista
    soma_valores = sum(valores)

    # Calcula os fatores de escala para manter a proporção
    fatores_escala = [valor_base / soma_valores * valor for valor in valores]

    # Arredonda os fatores de escala para números inteiros
    fatores_inteiros = [round(fator) for fator in fatores_escala[-1::-1]]

    # Ajusta para garantir que a soma seja igual ao valor base
    diferenca = valor_base - sum(fatores_inteiros)
    fatores_inteiros[-1] += diferenca

    return fatores_inteiros

In [ ]:
def calculate_best_starting_points_with_destination(latitudes, longitudes, demands, destination_latitude, destination_longitude, num_vehicles):
    # Índices possíveis para pontos de início
    possible_starts = list(range(len(latitudes)))

    # Combinar os índices dos pontos de início com a demanda total
    start_combinations = itertools.combinations(possible_starts, num_vehicles)

    best_starts_indices = min(start_combinations, key=lambda indices: sum(demands[i] for i in indices))

    # Obter as coordenadas correspondentes aos melhores pontos de início
    best_starting_points = [(latitudes[i], longitudes[i]) for i in best_starts_indices]

    # Adicionar o ponto de destino
    best_starting_points.append((destination_latitude, destination_longitude))

    return best_starting_points

In [ ]:
def capturate_start_points(list_to_start_points = list()):
    start_latitudes = list()
    start_longitudes = list()
    for points in list_to_stars_points:
        start_latitudes.append(point[0])
        start_longitudes.append(point[1])
    return start_latitudes, start_longitudes

In [ ]:
def plot_clusters_on_map_multicolor(lista_lat, lista_lon, chave_lat, chave_lon):
    # Criar conjunto de coordenadas chave para verificação
    coordenadas_chave = set(zip(chave_lat, chave_lon))

    # Filtrar coordenadas não chave que não estão nas coordenadas chave
    coordenadas_nao_chave_filtradas = [
        (lat, lon) for lat, lon in zip(lista_lat, lista_lon) if (lat, lon) not in coordenadas_chave
    ]

    # Separar coordenadas chave que não estão presentes nas outras listas
    coordenadas_chave_filtradas = [
        (lat, lon) for lat, lon in zip(chave_lat, chave_lon)
        if (lat, lon) not in set(zip(lista_lat, lista_lon))
    ]

    # Configurar o estilo do gráfico de dispersão
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 8))

    # Plotar os pontos não chave em escala de cinza
    plt.scatter(
        [lon for _, lon in coordenadas_nao_chave_filtradas],
        [lat for lat, _ in coordenadas_nao_chave_filtradas],
        color='lightgrey',
        label='Pontos não chave',
        s=100
    )

    # Plotar os pontos chave filtrados em vermelho
    plt.scatter(
        [lon for _, lon in coordenadas_chave_filtradas],
        [lat for lat, _ in coordenadas_chave_filtradas],
        color='red',
        label='Pontos chave',
        s=150,
        marker='X'
    )

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Pontos no Mapa')

    # Mostrar a legenda
    plt.legend()

    # Exibir o mapa
    plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_clusters_on_map_multicolor(lista_lat, lista_lon, chave_lat, chave_lon):
    # Criar um conjunto de coordenadas das listas de pontos não chave
    coordenadas_nao_chave = set(zip(lista_lat, lista_lon))

    # Filtrar os pontos chave que não estão presentes nas outras listas
    coordenadas_chave_validas = [(lat, lon) for lat, lon in zip(chave_lat, chave_lon) if (lat, lon) not in coordenadas_nao_chave]

    # Separar as coordenadas de pontos chave válidos
    chave_lat_valido, chave_lon_valido = zip(*coordenadas_chave_validas)

    # Configurar o estilo do gráfico de dispersão
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 8))

    # Plotar os pontos não chave em escala de cinza
    plt.scatter(lista_lon, lista_lat, color='lightgrey', label='Pontos não chave', s=100)

    # Plotar os pontos chave válidos em vermelho
    plt.scatter(chave_lon_valido, chave_lat_valido, color='red', label='Pontos chave', s=150, marker='X')

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Pontos no Mapa')

    # Mostrar a legenda
    plt.legend()

    # Exibir o mapa
    plt.show()

# Exemplo de uso:
# Suponha que você tenha as listas de coordenadas, por exemplo:
lista_lat = [40.7128, 34.0522, 37.7749, 34.0522]
lista_lon = [-74.0060, -118.2437, -122.4194, -118.2437]
chave_lat = [40.7128, 34.0522, 47.6062, 34.0522]
chave_lon = [-74.0060, -118.2437, -122.3321, -118.2437]

# Chamada da função
plot_clusters_on_map_multicolor(lista_lat, lista_lon, chave_lat, chave_lon)


### Mecânismo de inicilização das rotas

In [ ]:
def create_data_model(locations, num_routes, vehicle_capacities, qtds, destinations):
    data = {}
    data['locations'] = locations
    data['num_vehicles'] = num_routes
    data['vehicle_capacities'] = vehicle_capacities
    data['depot'] = 0  # Define o ponto de partida como o primeiro ponto da lista
    data['demands'] = qtds
    data['destinations'] = destinations
    return data

def get_best_starting_points(latitudes, longitudes, num_routes, vehicle_capacities, qtds, destinations):
    locations = [(lat, lon) for lat, lon in zip(latitudes, longitudes)]
    qtds = qtds
    data = create_data_model(locations, num_routes, vehicle_capacities, qtds, destinations)

    manager = pywrapcp.RoutingIndexManager(len(data['locations']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    best_starting_points = []
    visited_nodes = set()

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity')

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        
        return calculate_distance(data['locations'][from_node], data['locations'][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def destination_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['destinations'][from_node]

    destination_callback_index = routing.RegisterUnaryTransitCallback(destination_callback)

    routing.AddDimension(destination_callback_index, 1, 1, True, 'Destination')
    destination_dimension = routing.GetDimensionOrDie('Destination')

    for i in range(len(locations)):
        index = manager.NodeToIndex(i)
        destination_dimension.SetCumulVarSoftUpperBound(index, 1, 1)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None
    
    best_starting_points = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        node = manager.IndexToNode(index)
        best_starting_points.append((data['locations'][node], data['vehicle_capacities'][vehicle_id], data['destinations'][node]))

    return best_starting_points

# 1º Ramificação

## Caso initialize_starting_points seja False

Como a coluna 'start' existe no data frame, os potenciais pontos de partida das linhas ja foram definidos pelo usario

In [ ]:
if initialize_starting_points == False:
    starts = teste[teste['start']=='x']['index'].unique()
    n_routs_desirable = len(starts)
    
    # Definição dos pontos de latitude e longitudes onde as rotas devem iniciar
    # Essa é uma especificação do usuário deve ser respeitada
    # essas latitudes e longitudes serão plotadas no mapara para que o osuario possa ver
    inicial_latitudes = list(teste[teste['index'].isin(starts)]['lat'])
    inicial_longitudes = list(teste[teste['index'].isin(starts)]['long'])

# 2 º Ramificação

## Caso initialize_starting_points seja True

Como a coluna 'start' não existe no data frame, os potenciais pontos de partida das linhas devem ser definidos matematicamente

In [ ]:
import matplotlib.pyplot as plt

def plot_points_on_map(lista_lat, lista_lon):
    # Configurar o estilo do gráfico de dispersão
    plt.figure(figsize=(10, 8))

    # Plotar os pontos em escala de cinza
    plt.scatter(lista_lon, lista_lat, color='lightgrey', label='Pontos', s=100)

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Pontos no Mapa')

    # Mostrar a legenda
    plt.legend()

    # Exibir o mapa
    plt.show()

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(locations, num_routes, vehicle_capacities, qtds, destinations):
    data = {}
    data['locations'] = locations
    data['num_vehicles'] = num_routes
    data['vehicle_capacities'] = vehicle_capacities
    data['depot'] = 0  # Define o ponto de partida como o primeiro ponto da lista
    data['demands'] = qtds
    data['destinations'] = destinations
    return data

def calculate_distance(location1, location2):
    distance = haversine_distance(location1[0], location1[1],location2[0], location2[1])
    
    return distance

def get_best_starting_points(latitudes, longitudes, num_routes, vehicle_capacities, qtds, destinations):
    locations = [(lat, lon) for lat, lon in zip(latitudes, longitudes)]
    data = create_data_model(locations, num_routes, vehicle_capacities, qtds, destinations)

    manager = pywrapcp.RoutingIndexManager(len(data['locations']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity')

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return calculate_distance(data['locations'][from_node], data['locations'][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def destination_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['destinations'][from_node]

    destination_callback_index = routing.RegisterUnaryTransitCallback(destination_callback)

    routing.AddDimension(destination_callback_index, 1, 1, True, 'Destination')
    destination_dimension = routing.GetDimensionOrDie('Destination')

    for i in range(len(locations)):
        index = manager.NodeToIndex(i)
        destination_dimension.SetCumulVarSoftUpperBound(index, 1, 1)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    solution = routing.SolveWithParameters(search_parameters)

    
    if not solution:
        return None
    
    # Lógica para selecionar melhores pontos de partida
    best_starting_points = []
    visited_nodes = set()
    
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        node = manager.IndexToNode(index)
        best_starting_points.append(node)
        
    return best_starting_points
    
    

    
    
    

    

#     while len(best_starting_points) < data['num_vehicles']:
#         best_score = float('-inf')
#         best_node = None

#         for i in range(len(locations)):
#             if i not in visited_nodes:
#                 score = data['demands'][i] / calculate_distance(data['locations'][data['depot']], data['locations'][i])
#                 if score > best_score:
#                     best_score = score
#                     best_node = i

#         if best_node is not None:
#             visited_nodes.add(best_node)
#             node = manager.IndexToNode(best_node)
#             #best_starting_points.append([data['locations'][node], data['vehicle_capacities'][len(best_starting_points)], data['destinations'][node]])
#             best_starting_points.append(data['locations'][node])


#     return solution




In [ ]:
list_to_stars_points = list()
if initialize_starting_points == True:
    n = 18
    n_clusters = int(n**0.5)+1
    # Chamar a função para agrupar os dados em n grupos (de acoroo com o numero de rotas)
    clustered_df = cluster_data(teste[:], n_clusters)
    # Chamar a função para plotar os grupos no mapa
    x = plot_clusters_on_map(clustered_df, 'lat', 'long', 'cluster')
    y = clustered_df[['cluster', 'qtd', 'row']].groupby('cluster').sum(['qtd', 'row']).sort_values('qtd', ascending=False)
    y = y.reset_index()
    z = pd.merge(x, y, on='cluster')
    z= z.assign(lat1= -19.837913) 
    z= z.assign(long1= -40.082170)
    z['distance'] = haversine_distance(z['lat'], z['long'], z['lat1'], z['long1'])
    z = z.sort_values('qtd', ascending=False)
    base = n
    resultado = calcular_proporcionais(list(z['qtd']), base)
    z['n_routs'] = resultado
    best_starting_points = get_best_starting_points(teste['lat'], teste['long'], n,[46 for _ in range(n)], teste['qtd'], [0 for _ in range(n)])
    print(best_starting_points)
#     lats = list()
#     longs = list()
#     for value in best_starting_points:
#         lats.append(float(value[0]))
#         longs.append(float(value[1]))

# #     asdfsdf = pd.DataFrame({'lat': lats, 'long': longs})
# #     asdfsdf = asdfsdf.assign(profit='chave')
# #     teste = teste.merge(asdfsdf, on = ['lat', 'long'], how = 'left')
# #     teste['profit'] = teste['profit'].str.replace("", "sencudario")
#     plot_points_on_map(lats, longs)
#     print(len(lats))
#     print(len(longs))

In [ ]:
resultado

In [ ]:
teste[teste['index']==147]

In [ ]:
LATITUDES

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

def plot_map(primary_latitudes, primary_longitudes, secondary_coordinates):
    plt.figure(figsize=(10, 8))  # Define o tamanho da figura

    ax = plt.axes(projection=ccrs.PlateCarree())

    # Plota os pontos das listas principais em cinza
    ax.scatter(primary_longitudes, primary_latitudes, color='gray', label='Lista Principal')

    # Itera sobre as chaves das coordenadas secundárias
    for key, (latitudes, longitudes) in secondary_coordinates.items():
        # Plota os pontos correspondentes a cada chave em vermelho
        ax.scatter(longitudes, latitudes, label=f'Chave {key}', color='red')

    # Adiciona outros elementos ao mapa (limites, grade, etc.) conforme desejado
    ax.coastlines()
    ax.gridlines(draw_labels=True)

    # Adiciona legenda
    ax.legend()

    plt.show()

# Exemplo de uso:
# Suponha que você tenha as seguintes listas de coordenadas:
lista_principal_latitudes = [40.7128, 34.0522, 51.5074]
lista_principal_longitudes = [-74.0060, -118.2437, -0.1278]

# Dicionário de coordenadas secundárias com chaves menores
coordenadas_secundarias = {
    'A': ([48.8566, 37.7749, 31.9686], [2.3522, -122.4194, -99.9018]),
    'B': ([52.5200, 41.8781], [13.4050, -87.6298])
    # Pode adicionar mais chaves e suas coordenadas conforme necessário
}

# Chama a função para plotar os pontos
plot_map(lista_principal_latitudes, lista_principal_longitudes, coordenadas_secundarias)




In [1]:
from pytsp import atsp_tsp, run, dumps_matrix
import numpy as np

def get_best_starting_locations(distance_matrix, num_vehicles, destinos_indices, demands, vehicle_capacities):
    # Converter a matriz de distâncias em um formato aceito pelo PyTSP
    tsp_data = dumps_matrix(distance_matrix.tolist())

    # Configurar os parâmetros do problema do TSP
    tsp_params = {
        'problem': 'tsp',
        'solver': 'LKH',
        'tsp': tsp_data,
        'salesmen': num_vehicles,
        'start': destinos_indices,
        'depot': 0,  # Ponto de partida
        'return': True
    }

    # Resolver o problema do TSP com PyTSP
    tsp_results = run(tsp_params)

    # Recuperar os melhores locais para iniciar as rotas
    best_starting_locations = tsp_results['best_path']

    return best_starting_locations

# Exemplo de uso:
# Suponha que você tenha os dados de distância, número de veículos, destinos, demandas e capacidade dos veículos
# Vamos simular alguns dados aleatórios para demonstração
np.random.seed(42)
distance_matrix = np.random.randint(1, 100, (5, 5))  # Matriz de distâncias
num_vehicles = 2
destinos_indices = [1, 3]  # Índices dos destinos de acordo com a distance matrix
demands = [0, 5, 3, 2, 7]  # Demandas de cada ponto
vehicle_capacities = [10, 15]  # Capacidade de cada veículo

# Obter os melhores locais para iniciar as rotas
best_starting_locations = get_best_starting_locations(distance_matrix, num_vehicles, destinos_indices, demands, vehicle_capacities)

print("Melhores locais para iniciar as rotas:", best_starting_locations)


ImportError: cannot import name 'atsp_tsp' from 'pytsp' (C:\Users\MarkosPereira.exc\Anaconda3\lib\site-packages\pytsp\__init__.py)

In [ ]:
!pip install --force-reinstall numpy

In [3]:
import pytsp

In [6]:
print(help(pytsp.))

Help on package pytsp:

NAME
    pytsp - Top-level package for pytsp.

PACKAGE CONTENTS
    branch_and_bound_tsp
    branch_and_bound_tsp_dfs
    christofides_tsp
    constants
    data_structures (package)
    genetic_algorithms (package)
    had_tsp
    k_opt_tsp
    lin_kerdighan_tsp
    nearest_neighbor_tsp
    permutations_tsp
    pytsp
    utils

DATA
    __email__ = 'gazda.matej@gmail.com'
    name = 'pytsp'

VERSION
    0.1.0

AUTHOR
    BraveDistribution

FILE
    c:\users\markospereira.exc\anaconda3\lib\site-packages\pytsp\__init__.py


None


In [10]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

def solve_vrp(distance_matrix, num_vehicles, destinos_indices, demands, vehicle_capacities):
    # Função para criar o modelo do VRP
    def create_data_model():
        data = {}
        data['distance_matrix'] = distance_matrix
        data['num_vehicles'] = num_vehicles
        data['demands'] = demands
        data['vehicle_capacities'] = vehicle_capacities
        data['depot'] = 0  # Ponto de partida é o primeiro índice (0)

        return data

    def distance_callback(from_index, to_index):
        return data['distance_matrix'][manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    data = create_data_model()

    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity')

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        best_starting_locations = []
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            node = manager.IndexToNode(index)
            best_starting_locations.append(node)
        return best_starting_locations
    else:
        return None

# Exemplo de uso
# Substitua distance_matrix, num_vehicles, destinos_indices, demands e vehicle_capacities pelos seus próprios dados
distance_matrix = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])
num_vehicles = 2
destinos_indices = [1, 3]
demands = [0, 5, 3, 2]
vehicle_capacities = [10, 15]

best_starting_locations = solve_vrp(distance_matrix, num_vehicles, destinos_indices, demands, vehicle_capacities)
print("Melhores locais para iniciar as rotas:", best_starting_locations)


Melhores locais para iniciar as rotas: [0, 0]


In [14]:
np.random.seed(42)
distance_matrix = np.random.randint(1, 100, (10, 10))  # Matriz de distâncias
print(distance_matrix)
num_vehicles = 3
destinos_indices = [9, 9]  # Índices dos destinos de acordo com a distance matrix
demands = [9, 5, 3, 2, 7]  # Demandas de cada ponto
vehicle_capacities = [10, 15, ]  # Capacidade de cada veículo
print(solve_vrp(distance_matrix, num_vehicles, destinos_indices, demands, vehicle_capacities))

[[52 93 15 72 61 21 83 87 75 75]
 [88 24  3 22 53  2 88 30 38  2]
 [64 60 21 33 76 58 22 89 49 91]
 [59 42 92 60 80 15 62 62 47 62]
 [51 55 64  3 51  7 21 73 39 18]
 [ 4 89 60 14  9 90 53  2 84 92]
 [60 71 44  8 47 35 78 81 36 50]
 [ 4  2  6 54  4 54 93 63 18 90]
 [44 34 74 62 14 95 48 15 72 78]
 [87 62 40 85 80 82 53 24 26 89]]
[0, 0]
